In [1]:
!pip install pronouncing

In [2]:
!pip install Metaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13902 sha256=018fc34d9085ec7a233d64dee2185133aeeb18655454a66ca82ce9bf5d9f2cc4
  Stored in directory: /home/byczong/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built Metaphone


In [60]:
import pronouncing
from metaphone import doublemetaphone
import re

### Finding rhymes

- Goal: Find set of words (unigrams) that rhyme with a given word

- Rough algorithm plan:
    - Input: `s`: str
    - Expand any abbreviations, symbols and numbers in `s` ('mr' -> 'mister', '%' -> 'percent')
    - Convert with the chosen phonetic algorithm (Soundex, Metaphone ...) to phonetic form (as pronounced in English)
    - Find similar, existing words (from unigrams file) based on matching suffixes (need to choose the appropriate suffix lenght)

- https://en.wikipedia.org/wiki/Soundex
    - (1918)
- https://en.wikipedia.org/wiki/Metaphone
    - Metaphone (1990)
    - Double Metaphone (2000)
    - Metaphone 3 (2009) - commercial product

**CMU ARPABET dictionary**

In [4]:
pronouncing.phones_for_word("sighs")

['S AY1 Z']

In [5]:
pronouncing.rhymes("failings")

['mailings', 'railings', 'tailings']

**Double Metaphone**

- does not take vowels (other than the first character) into account :(
- it can return both a primary and a secondary code for a string

In [44]:
def comp(t1, t2):
    print(f'{t1}\n{t2}')

In [45]:
comp(doublemetaphone("shell"), doublemetaphone("smell"))

('XL', '')
('SML', 'XML')


In [46]:
comp(doublemetaphone("generate"), doublemetaphone("degenerate"))

('JNRT', 'KNRT')
('TJNRT', 'TKNRT')


Example "rhyme representation" algorithm:

- Input string s
- m = doublemetaphone(s)
- result = insert the vowels from s inside m  # (todo: how)
- in order to find set of rhymes match same suffixes # (todo: how long should the suffix be)

In [55]:
comp('generate',
     doublemetaphone('generate')[0])
'TJeNeRaTe'

generate
JNRT


'TJeNeRaTe'

In [58]:
comp('failings',
     doublemetaphone('failings')[0])
'FaiLiNKS'

failings
FLNKS


'FaiLiNKS'

In [73]:
vowels = 'aeiouy'

def encode_into_rhyme_repr(s: str) -> str:
    # get the doublemetaphone repr of s
    m = doublemetaphone(s)[0]
    
    # find the position (relative from the end) of the second vowel in s 
    # (if the vowel is not the first letter and if there are at least 2 vowels in s)
    
    # addtionally:
    
    # ends with a vowel => find the position of the second vowel
    # no vowel in s[-2:] => find the position of the first vowel
    total_vowel_count = sum([c in vowels for c in s])

    pos = -1
    vowel_count = 0
    for i, c in enumerate(reversed(m)):
        pos = -1 - i
        print(m[pos])

In [74]:
encode_into_rhyme_repr('byczong')

K
N
S
P
